In [1]:
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *

In [2]:
# 辅助函数
# 栅格转面
def raster_to_polygon(in_raster, out_polygon, field_name):
    """栅格转面"""
    in_raster = arcpy.sa.Int(in_raster)
    output_coordinate_system = arcpy.Describe(in_raster).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system):
        arcpy.conversion.RasterToPolygon(in_raster, out_polygon, "SIMPLIFY", f"{field_name}", "SINGLE_OUTER_PART", None)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

In [3]:
# 构建转换字典
table_path = r"F:\cache_data\frequency_filter\dy\scaler_csv\dy_stander_filter_all_type.csv"

In [4]:
df = pd.read_csv(table_path)

In [5]:
# 对每个唯一的土种编号生成一个字典
def aggregate_rows(group):
    records = group[['TL', 'YL', 'TS']].to_dict('records')
    # 假设每个土种编号下的记录是相同的，这里我们取第一个记录
    return records[0] if records else {}

# 使用groupby按照`土种编号`聚合，并应用自定义函数
soil_dict = df.groupby('TZ').apply(aggregate_rows).to_dict()

In [6]:
tz_dict = {0: ['中白鳝泥田'], 1: ['偏沙土'], 2: ['冷水田'], 3: ['冷沙土'], 4: ['冷沙田'], 5: ['冷浸田'], 6: ['厚腐殖层中土层硅质黄壤'], 7: ['厚腐殖层中土层黄色石灰土'], 8: ['厚腐殖层薄土层硅质黄壤'], 9: ['大泥土'], 10: ['大泥田'], 11: ['大眼泥田'], 12: ['大眼黄泥田'], 13: ['小泥土'], 14: ['小黄泥土'], 15: ['小黄泥田'], 16: ['岩泥'], 17: ['干鸭屎泥田'], 18: ['扁沙泥田'], 19: ['死黄泥土'], 20: ['死黄泥田'], 21: ['浅足烂泥田'], 22: ['深足烂泥田'], 23: ['湿鸭屎泥田'], 24: ['潮板沙田'], 25: ['潮沙泥田'], 26: ['潮沙田'], 27: ['潮油沙泥田'], 28: ['潮泥田'], 29: ['火石沙土'], 30: ['烂锈田'], 31: ['熟鸭屎泥田'], 32: ['白云沙土'], 33: ['白沙土'], 34: ['白沙泥田'], 35: ['石碴子土'], 36: ['砾质紫沙土'], 37: ['紫沙土'], 38: ['紫色土'], 39: ['苦大泥田'], 40: ['薄腐殖层中土层山地灌丛草甸土'], 41: ['薄腐殖层中土层次生黄色石灰土'], 42: ['薄腐殖层中土层硅铁质黄壤'], 43: ['薄腐殖层中土层硅铝质黄壤'], 44: ['薄腐殖层中土层紫色土'], 45: ['薄腐殖层中土层酸性紫色土'], 46: ['薄腐殖层中土层铁铝质黄壤'], 47: ['薄腐殖层中土层黄壤性土'], 48: ['薄腐殖层中土层黄色石灰土'], 49: ['薄腐殖层中土层黑色石灰土'], 50: ['薄腐殖层厚土层次生黄色石灰土'], 51: ['薄腐殖层厚土层淋溶黄色石灰土'], 52: ['薄腐殖层厚土层硅质黄壤'], 53: ['薄腐殖层厚土层硅铁质黄壤'], 54: ['薄腐殖层厚土层硅铝质黄壤'], 55: ['薄腐殖层厚土层紫色土'], 56: ['薄腐殖层厚土层铁铝质黄壤'], 57: ['薄腐殖层厚土层黄色石灰土'], 58: ['薄腐殖层厚土层黑色石灰土'], 59: ['薄腐殖层薄土层硅质黄壤'], 60: ['薄腐殖层薄土层硅铁质黄壤'], 61: ['薄腐殖层薄土层硅铝质黄壤'], 62: ['薄腐殖层薄土层铁质黄壤'], 63: ['薄腐殖层薄土层铁铝质黄壤'], 64: ['薄腐殖层薄土层黄色石灰土'], 65: ['薄腐殖层薄土层黑色石灰土'], 66: ['薄腐殖薄土层山地灌丛草甸土'], 67: ['重白胶泥田'], 68: ['重白鳝泥田'], 69: ['钙石子土'], 70: ['铅毒田'], 71: ['锈水田'], 72: ['雾黄泥田'], 73: ['马血泥田'], 74: ['黄大泥田'], 75: ['黄沙泥土'], 76: ['黄沙泥田'], 77: ['黄油沙泥田'], 78: ['黄泥土'], 79: ['黄泥田'], 80: ['黄石砂田'], 81: ['黄胶泥土'], 82: ['黄胶泥田'], 83: ['黑潮泥田'], 84: ['龙凤大眼泥田']}

In [7]:
result_dict = {}
for one_key in tz_dict:
    result_dict[one_key] = {'TZ':tz_dict[one_key][0],'TL':soil_dict[tz_dict[one_key][0]]['TL'],'TS':soil_dict[tz_dict[one_key][0]]['TS'],'YL':soil_dict[tz_dict[one_key][0]]['YL']}

In [8]:
result_dict

{0: {'TZ': '中白鳝泥田',
  'TL': '水稻土',
  'TS': '白泥田A—P E—E(C)或A—P—E(C)',
  'YL': '侧渗型水稻土'},
 1: {'TZ': '偏沙土', 'TL': '黄壤', 'TS': '石砂土', 'YL': '黄泥土'},
 2: {'TZ': '冷水田', 'TL': '水稻土', 'TS': '冷水田', 'YL': '潴育型水稻土'},
 3: {'TZ': '冷沙土', 'TL': '黄壤', 'TS': '石砂土', 'YL': '黄泥土'},
 4: {'TZ': '冷沙田', 'TL': '水稻土', 'TS': '冷浸田APG或APWC', 'YL': '潜育型水稻土'},
 5: {'TZ': '冷浸田', 'TL': '水稻土', 'TS': '冷浸田APG或APWC', 'YL': '潜育型水稻土'},
 6: {'TZ': '厚腐殖层中土层硅质黄壤', 'TL': '黄壤', 'TS': '硅质黄壤', 'YL': '黄壤'},
 7: {'TZ': '厚腐殖层中土层黄色石灰土', 'TL': '石灰土', 'TS': '黄色石灰土', 'YL': '黄色石灰土'},
 8: {'TZ': '厚腐殖层薄土层硅质黄壤', 'TL': '黄壤', 'TS': '硅质黄壤', 'YL': '黄壤'},
 9: {'TZ': '大泥土', 'TL': '石灰土', 'TS': '大土泥', 'YL': '大土泥'},
 10: {'TZ': '大泥田', 'TL': '水稻土', 'TS': '淹育型石灰性大泥田APC', 'YL': '淹育型水稻土'},
 11: {'TZ': '大眼泥田', 'TL': '水稻土', 'TS': '潴育型石灰性大眼泥田ARWC或APBC', 'YL': '潴育型水稻土'},
 12: {'TZ': '大眼黄泥田', 'TL': '水稻土', 'TS': '潴育型石灰性大眼泥田ARWC或APBC', 'YL': '潴育型水稻土'},
 13: {'TZ': '小泥土', 'TL': '石灰土', 'TS': '小泥土', 'YL': '大土泥'},
 14: {'TZ': '小黄泥土', 'TL': '黄壤', 'TS': '黄泥土', 'YL': 

In [9]:
raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb\prediction_sioltype_CATB_20240226_smooth'
out_polygon_path = r'D:\ArcGISProjects\workspace\shbyq\soil_type_result.gdb\soiltype_dy\soiltype_prediction_CATB_20240226'
field_name = 'Value'

In [10]:
# 栅格转面
raster_to_polygon(raster_path, out_polygon_path, field_name)

In [19]:
code_block =f"""
def get_soil_class(soil_code):
    soil_dict = {result_dict}
    TZ = soil_dict[soil_code]['TZ']
    TL = soil_dict[soil_code]['TL']
    YL = soil_dict[soil_code]['YL']
    TS = soil_dict[soil_code]['TS']
    result_list = [TZ,TS,YL,TL]
    return result_list[0]
"""
field_name = 'TZ'

In [12]:
expression = "get_soil_class(!gridcode!)"

In [13]:
# 增加字段
for one_field in ['TL','YL','TS','TZ']:
    add_field(out_polygon_path, one_field, 'TEXT')

In [20]:
# 计算字段
arcpy.CalculateField_management(in_table=out_polygon_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)
    

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\soil_type_result.gdb\\soiltype_dy\\soiltype_prediction_CATB_20240226'>